In [ ]:
"""
    AveragedPowerspectrum{T<:AbstractFloat}

A type representing an averaged power spectrum computed from time series data.

# Fields
- `freqs::Vector{T}`: Frequency bins
- `power::Vector{T}`: Power values corresponding to each frequency bin
- `power_err::Vector{T}`: Error estimates for each power value
"""
struct AveragedPowerspectrum{T<:AbstractFloat}
    freqs::Vector{T}
    power::Vector{T}
    power_err::Vector{T}
end

"""
    powerspectrum(events::EventList, dt::Real, segment_size::Real; 
                 norm::String="frac", 
                 use_common_mean::Bool=true) -> AveragedPowerspectrum

Compute an averaged power spectrum from an EventList.

# Arguments
- `events`: EventList containing time series data
- `dt`: Time resolution for binning
- `segment_size`: Size of segments to average (in seconds)
- `norm`: Normalization method ("frac", "leahy", "abs", or "none")
- `use_common_mean`: Whether to use common mean across all segments
"""
function powerspectrum(events::EventList{T}, 
                      dt::Real, 
                      segment_size::Real;
                      norm::String="frac",
                      use_common_mean::Bool=true) where T
    
    # Calculate number of bins per segment
    n_bin = round(Int, segment_size / dt)
    adjusted_dt = segment_size / n_bin

    # Split events into segments
    segments = segment_events(events, segment_size, dt)
    
    if isempty(segments)
        return AveragedPowerspectrum{T}(T[], T[], T[])
    end

    # Calculate power spectra for each segment
    powers = zeros(T, length(segments), n_bin ÷ 2 + 1)
    n_segments = 0
    total_counts = zero(T)

    for (i, segment) in enumerate(segments)
        if !isempty(segment)
            powers[i, :], counts = compute_segment_power(segment, n_bin, adjusted_dt)
            total_counts += counts
            n_segments += 1
        end
    end

    # Average the power spectra
    if n_segments == 0
        return AveragedPowerspectrum{T}(T[], T[], T[])
    end

    avg_power = vec(mean(powers[1:n_segments, :], dims=1))
    freqs = fftfreq(n_bin, adjusted_dt)[1:(n_bin ÷ 2 + 1)]
    
    # Apply normalization
    mean_counts = total_counts / (n_segments * n_bin)
    normalized_power = normalize_power(avg_power, adjusted_dt, n_bin, mean_counts, norm)
    
    # Calculate errors
    power_err = normalized_power ./ sqrt(n_segments)

    return AveragedPowerspectrum{T}(freqs, normalized_power, power_err)
end

"""
    segment_events(events::EventList, segment_size::Real, dt::Real) -> Vector{Vector{Int}}

Split event times into segments and bin them.
"""
function segment_events(events::EventList, segment_size::Real, dt::Real)
    start_time = minimum(events.times)
    end_time = maximum(events.times)
    
    # Calculate number of complete segments
    n_segments = floor(Int, (end_time - start_time) / segment_size)
    if n_segments == 0
        return Vector{Int}[]
    end
    
    n_bins = round(Int, segment_size / dt)
    segments = [Int[] for _ in 1:n_segments]
    
    # Bin events into segments
    for t in events.times
        seg_idx = floor(Int, (t - start_time) / segment_size) + 1
        if 1 ≤ seg_idx ≤ n_segments
            bin_idx = floor(Int, ((t - start_time) % segment_size) / dt) + 1
            if 1 ≤ bin_idx ≤ n_bins
                push!(segments[seg_idx], bin_idx)
            end
        end
    end
    
    return segments
end

"""
    compute_segment_power(binned_events::Vector{Int}, n_bins::Int, dt::Real) -> Tuple{Vector{Float64}, Int}

Compute power spectrum for a single segment of binned events.
"""
function compute_segment_power(binned_events::Vector{Int}, n_bins::Int, dt::Real)
    counts = zeros(Int, n_bins)
    for bin in binned_events
        counts[bin] += 1
    end
    
    ft = fft(counts)
    power = abs2.(ft[1:(n_bins ÷ 2 + 1)])
    
    return power, sum(counts)
end

"""
    normalize_power(power::AbstractVector, dt::Real, n_bins::Int, mean_counts::Real, norm::String) -> Vector{Float64}

Apply normalization to power spectrum.
"""
function normalize_power(power::AbstractVector, dt::Real, n_bins::Int, mean_counts::Real, norm::String)
    normalized = copy(power)
    
    if norm == "frac"
        normalized ./= (mean_counts^2 * n_bins)
    elseif norm == "leahy"
        normalized .*= (2 / mean_counts)
    elseif norm == "abs"
        normalized .*= (2 * dt / n_bins)
    elseif norm != "none"
        throw(ArgumentError("Unknown normalization method: $norm"))
    end
    
    return normalized
end

"""
    fftfreq(n::Int, d::Real=1.0) -> Vector{Float64}

Calculate the FFT frequencies for a given number of points and sampling interval.
"""
function fftfreq(n::Int, d::Real=1.0)
    val = 1.0 / (n * d)
    N = (n-1) ÷ 2 + 1
    freq = zeros(n)
    freq[1:N] = 0:(N-1)
    freq[N+1:end] = (-n ÷ 2):-1
    return freq .* val
end